In [4]:
# install the needed libraries
#!pip uninstall tensorflow -y
#!pip install tensorflow-macos==2.10.0
#!pip install google-cloud


In [47]:
#Import the Libraries
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
import pandas as pd
import random

Due to format issues, we decided to drop the photo2 and sketch2 folders (98 images each)

# Preprocessing Dataset 1

In [48]:
# Define photo1 and sketch 1 path names 
photo_path_1 =os.path.join(os.getcwd(),"..","data","raw_data","photo","photo1")
sketch_path_1 =os.path.join(os.getcwd(),"..","data","raw_data","sketch","sketch1")

#Create two lists of the filenames in the corresponding paths and sort them
photo_list_1 = os.listdir(photo_path_1)
if '.DS_Store' in photo_list_1 : 
    photo_list_1.remove('.DS_Store')
photo_list_1.sort()

sketch_list_1 = os.listdir(sketch_path_1)
if '.DS_Store' in sketch_list_1 : 
    sketch_list_1.remove('.DS_Store')
sketch_list_1.sort()

In [49]:
#check the lengths of the 2 lists
len(photo_list_1), len(sketch_list_1)

(1529, 1529)

In [50]:
#resize Photos and store them in the photo_preproc folder

dest_folder_photos = os.path.join(os.getcwd(),"..","data","raw_data","photo")

for i in range(len(photo_list_1)):
    photo=Image.open(os.path.join(photo_path_1,photo_list_1[i]))
    resized_photo=photo.resize((256,256))
    resized_photo.save(os.path.join(os.getcwd(),dest_folder_photos,f"photo{str(i+1).rjust(4,'0')}.jpg"))

In [51]:
#resize Sketches and store them in the sketch_preproc folder

dest_folder_sketches = os.path.join(os.getcwd(),"..","data","raw_data","sketch")

for j in range(len(sketch_list_1)):
    sketch=Image.open(os.path.join(sketch_path_1,sketch_list_1[j]))
    resized_sketch=sketch.resize((256,256))
    resized_sketch.save(os.path.join(os.getcwd(),dest_folder_sketches,f"sketch{str(j+1).rjust(4,'0')}.jpg"))

# Preprocessing Dataset 2

In [52]:
# Define photo1 and sketch 1 path names 
photo_path_2 =os.path.join(os.getcwd(),"..","data","raw_data","photo","photo2")
sketch_path_2 =os.path.join(os.getcwd(),"..","data","raw_data","sketch","sketch2")

#Create two lists of the filenames in the corresponding paths and sort them
photo_list_2 = os.listdir(photo_path_2)
if '.DS_Store' in photo_list_2 : 
    photo_list_2.remove('.DS_Store')
photo_list_2.sort()

sketch_list_2 = os.listdir(sketch_path_2)
if '.DS_Store' in sketch_list_2 : 
    sketch_list_2.remove('.DS_Store')
sketch_list_2.sort()

In [53]:
#check the lengths of the 2 lists
len(photo_list_2), len(sketch_list_2)

(477, 477)

In [54]:
#resize Photos and store them in the photo_preproc folder

dest_folder_photos = os.path.join(os.getcwd(),"..","data","raw_data","photo")

for i in range(len(photo_list_2)):
    photo=Image.open(os.path.join(photo_path_2,photo_list_2[i]))
    resized_photo=photo.resize((256,256))
    resized_photo.save(os.path.join(os.getcwd(),dest_folder_photos,f"photo{str(i+1530).rjust(4,'0')}.jpg"))

In [55]:
#resize Sketches and store them in the sketch_preproc folder

dest_folder_sketches = os.path.join(os.getcwd(),"..","data","raw_data","sketch")

for j in range(len(sketch_list_2)):
    sketch=Image.open(os.path.join(sketch_path_2,sketch_list_2[j]))
    resized_sketch=sketch.resize((256,256))
    resized_sketch.save(os.path.join(os.getcwd(),dest_folder_sketches,f"sketch{str(j+1530).rjust(4,'0')}.jpg"))

# Train test split

In [56]:
# USEFUL FUNCTIONS FOR TRAIN TEST SPLIT

#Keep only the image names and gets rid of folder names etc
def clean_namelist(raw_namelist):
#clean the .jpg name list and sort it
    clean_name_list = list()
    for x in raw_namelist:
        if ".jpg" in x.lower():
            clean_name_list.append(x)
    clean_name_list.sort()
    return clean_name_list

#Create a dictionary from two lists of equal length
def get_photo_sketch_dict(photo_list,sketch_list):
    """Takes 2 lists of matching length as input and returns a dictionary with the
    first list as key and the second list as value (in corresponding order"""

    if len(photo_list)==len(sketch_list):
        return {photo_list[i]:sketch_list[i] for i in range(len(photo_list))}
    else :
        print("The length of the sketch list must match the length of the photo list! Please check the data folders.")
        return None

#sample photos from a photo list
def get_photo_sample(photo_list, n_samples):

    photo_sample = random.sample(photo_list,int(n_samples))
    return photo_sample

#Sample corresponding sketches from a photo sample and a translation dict
def get_sketch_sample (photo_sample,translation_dict):

    sketch_sample = [translation_dict[photo] for photo in photo_sample]
    return sketch_sample

In [57]:
# Get the clean lists for photos and sketches

photo_list_clean=clean_namelist(os.listdir(dest_folder_photos))
sketch_list_clean=clean_namelist(os.listdir(dest_folder_sketches))

len(photo_list_clean),len(sketch_list_clean)

(2006, 2006)

In [58]:
#create a translation dictionary from the two clean lists
translation_dict=get_photo_sketch_dict(photo_list_clean,sketch_list_clean)

# sample the 400 photos
photo_test=get_photo_sample(photo_list_clean,400)
photo_test.sort()
#get the sample of corresponding sketches
sketch_test=get_sketch_sample(photo_test,translation_dict)
sketch_test.sort()

In [59]:
#get the train photos and sketches by difference

photo_train=[photo for photo in photo_list_clean if photo not in photo_test]
photo_train.sort()

sketch_train=get_sketch_sample(photo_train,translation_dict)
sketch_train.sort()

In [60]:
#check the length of all the lists
print("Train:",len(photo_train),len(sketch_train))
print("Test:",len(photo_test),len(sketch_test))

Train: 1606 1606
Test: 400 400


In [61]:
#Define the destination folders
train_photo_path=os.path.join(os.getcwd(),"..","data","preproc_data","train","photo")
train_sketch_path=os.path.join(os.getcwd(),"..","data","preproc_data","train","sketch")
test_photo_path=os.path.join(os.getcwd(),"..","data","preproc_data","test","photo")
test_sketch_path=os.path.join(os.getcwd(),"..","data","preproc_data","test","sketch")

In [62]:
for photo in photo_train:
    shutil.move(os.path.join(dest_folder_photos,photo),os.path.join(train_photo_path,photo))
    
for photo in photo_test:
    shutil.move(os.path.join(dest_folder_photos,photo),os.path.join(test_photo_path,photo))
    
for sketch in sketch_train:
    shutil.move(os.path.join(dest_folder_sketches,sketch),os.path.join(train_sketch_path,sketch))   

for sketch in sketch_test:
    shutil.move(os.path.join(dest_folder_sketches,sketch),os.path.join(test_sketch_path,sketch))  

In [68]:
#To remove the files

photo_train=os.listdir(train_photo_path)
photo_test=os.listdir(test_photo_path)
sketch_train=os.listdir(train_sketch_path)
sketch_test=os.listdir(test_sketch_path)

for photo in photo_train:
    os.remove(os.path.join(train_photo_path,photo))

for photo in photo_test:
    os.remove(os.path.join(test_photo_path,photo))

for sketch in sketch_train:
    os.remove(os.path.join(train_sketch_path,sketch))
    
for sketch in sketch_test:
    os.remove(os.path.join(test_sketch_path,sketch))

In [69]:
dest_folder_photos = os.path.join(os.getcwd(),"..","data","raw_data","photo")
dest_folder_sketches = os.path.join(os.getcwd(),"..","data","raw_data","sketch")

photo_list_clean=clean_namelist(os.listdir(dest_folder_photos))
sketch_list_clean=clean_namelist(os.listdir(dest_folder_sketches))

for photo,sketch in zip(photo_list_clean,sketch_list_clean):
    os.remove(os.path.join(dest_folder_photos,photo))
    os.remove(os.path.join(dest_folder_sketches,sketch))

    

In [45]:
round(2006*0.2)

401